In [ ]:
"""Import Data"""
import pandas as pd
import numpy as np

all_features_df = pd.read_csv('')

# Predictive Modelling (in progess)


**Exploration:** <br>
Data Visualization: t-SNE <br>
Data shortening: PCA <br>
Unsupervised Clustering: Heirarchical Clustering, KMeans, DTW, Spectral Clustering<br>
Supervised Classification: Deep Learning<br>


# Initial Visualization:

## 0.1 t-SNE (very slow- halted)

In [3]:
# from sklearn.manifold import TSNE
# tsne = TSNE(learning_rate = 1000,n_components=2) # change learning rate from 50 to 200. good --> not all are bunched together
## 0.1 Visulization: t-SNE (very slow- halted)
# tsne_features = tsne.fit_transform(stocks_df)

# xs = tsne_features[:,0]
# ys = tsne_features[:,1]
# plt.scatter(xs,ys)
# plt.show()

# Extracting Principal Components

## 1.1 PCA

# Unsupervised Clustering

## 2.1 Hierarchical Clustering

Survey of Time Series Clustering Algos: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.115.6594&rep=rep1&type=pdf



In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import normalize
mergings = linkage(samples, method='complete')

# Plot the dendrogram, using varieties as labels
dendrogram(mergings,
           labels=varieties,
           leaf_rotation=90,
           leaf_font_size=6,
)
plt.show()

## 2.2 KMeans

In [ ]:
# from sklearn.preprocessing import Normalizer, StandardScaler
# from sklearn.cluster import KMeans
# from sklearn.pipeline import Pipeline


# #n_clusters = elbow of KMeans.interia_ plot n),  industries ~ 11
# stocks_df.dropna(inplace = True)
 
# kmeans_pipeline = Pipeline(steps = [('nrml',Normalizer()),('clf',KMeans(n_clusters = 11))]) #('norm', Normalizer())
# kmeans_pipeline.fit(stocks_df)
# labels = kmeans_pipeline.predict(stocks_df)
# stocks_df.describe()
# plt.plot(labels)
# plt.show()
# print(labels)

# # Cross Tabulation to see effectiveness of split
# df_res = pd.DataFrame({'labels': labels,'companies': stocks_df.index})
# res = pd.crosstab(df_res['labels'], df_res['companies'])
# print(res)

# # Another trial
# # df = stored_df.reset_index().pivot(index = 'date', columns='Name')
# # df.columns = df.columns.swaplevel(0, 1)
# # df.sort_index(axis=1, level=0, inplace=True)
# # df

## 2.3 Dynamic Time Warping (DTW)

Used instead of K-Means as a way to cluster time series data.
https://www.cs.ucr.edu/~eamonn/time_series_data_2018/

# Supervised Clustering

Based on partial target data from https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

## 3. Deep Learning